## Setup

Make sure you're running the latest Julia! You will need to use an app like "jill" to get this on Ubuntu (the system packages are old).

To use this notebook, run the following to install the dependencies. Note, the first time you use anything (library, function, etc), Julia compiles it. This means that writing `using CUDA` takes ~1m42s on my workstation the first time I run it. Be warned!

In [10]:
using Pkg 
Pkg.add(["CUDA", "IntervalOptimisation", "Symbolics", "IntervalArithmetic","JuMP","EAGO"])

   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


...

  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## First adventures

Julia is like python but also like C++. It also has tons of built in math, like numpy-style array operations are in by default

In [11]:
#A generic function
f(x) = 2*x + x^2 + 3

display(f(2))

#Make a vector
v = [1.0, 2,3]

f.(v)#The dot is special when it appears before an operator (like the call operator here). It generally means do this elementwise

11

3-element Vector{Float64}:
  6.0
 11.0
 18.0

Note that Julia determined the type (Float64) from the literal's used (in this case the 1.0). It naturally has mathematical operations defined too, like vector addition. I can even do a dot product   

In [13]:
using LinearAlgebra
dot(v,v)

14.0

This isn't sexy enough. Write `v ` then `\cdot` and press tab to autocomplete (yes latex is the way these are defined), then put `v` and you get a dot product 

In [15]:
v ⋅ v

3-element Vector{Float64}:
 0.0
 0.0
 0.0

That's right, we have unicode, and a lot of it is plumbed into real mathematical definitions. We can even use it in our variable names to give them subscripts by typing `C\_2⭾`

In [16]:
C₂ = 1
π

π = 3.1415926535897...

Constants like pi are already there, but even these have been improved. What exactly is that pi?

In [13]:
typeof(π)

Irrational{:π}

It understands Irrationals! This means its an arbitrary precision element too as it retains its symbolic nature.

In [14]:
display(2π)
typeof(2π)

6.283185307179586

Float64

But it often falls back to float64 (double) by default, which is what we want MOST of the time for speed. But we can of course request arbitrary precision if we want... Here's a float256 (by default)

In [18]:
BigFloat(2)*π

6.283185307179586476925286766559005768394338798750211641949889184615632812572396

Enough digits yet? Lets look at functions again...

In [19]:
f(x) = 2 + 5sin(x)

f (generic function with 1 method)

What is f?

In [20]:
code_lowered(f)

1-element Vector{Core.CodeInfo}:
 CodeInfo(
1 ─ %1 = Main.sin(x)
│   %2 = 5 * %1
│   %3 = 2 + %2
└──      return %3
)

Cool! I can see the code! Hmm, looks a little generic and slow. What if I put the type information in, do I see more code?

In [21]:
@code_typed f(2.0)

CodeInfo(
1 ─ %1 = invoke Main.sin(x::Float64)::Float64
│   %2 = Base.mul_float(5.0, %1)::Float64
│   %3 = Base.add_float(2.0, %2)::Float64
└──      return %3
) => Float64

Its compiling to some specialised operations for floats where its tracking the types, the values, and the arguments. Can we keep going till we hit bare metal (assembly)? We can with @code_native, but @code_llvm is a more human-readable representation.

In [24]:
code_llvm(f, (Float64,))

;  @ /home/mjki2mb2/testJulia/test.ipynb:1 within `f`
define double @julia_f_3313(double %0) #0 {
top:
  %1 = call double @j_sin_3315(double %0) #0
; ┌ @ promotion.jl:389 within `*` @ float.jl:385
   %2 = fmul double %1, 5.000000e+00
; └
; ┌ @ promotion.jl:388 within `+` @ float.jl:383
   %3 = fadd double %2, 2.000000e+00
; └
  ret double %3
}


Awesome, llvm's Intermediate Representation, the last stage before assembly! All created whenever we use the function, but not before! So we always get compiled, fully optimised code.

So far, this is just C++ with python syntax. What makes Julia special? Lets take a look at the source code of `cos`

In [25]:
@which cos(0.5)

cos(x::T) where T<:Union{Float32, Float64} in Base.Math at special/trig.jl:98

You can even edit the source code on your computer using @edit, or scroll it using @less.

What you should see is some Horners' method implementations. But Julia is aware of this code right now, while compiling your code. It can optimise. Lets check it out

In [26]:
g() = cos(0.5)

@code_typed g()

CodeInfo(
1 ─     return 0.8775825618903728
) => Float64

While this might seem to be an obvious optimisation to you now, its not obvious for interpreted languages like python. Also, this is happening for ALL your code. Basically anything that doesn't change, is evaluated for free. Julia, thanks to LLVM, can deeply inspect your code and optimise static branches.

What I'm excited about is this is always available. Consider what has to happen for below to be true

In [34]:
fᵣ = Meta.parse("cos(0.5)")
fᵣ.args[1] = :sin
fᵣ

:(sin(0.5))

In [29]:
x = eval(fᵣ)

0.8775825618903728

# CUDA
We're always writing fairly abstract functions that then get adapted to native code only when run when types are known. Changing the type can change more than just the instructions, it can change where all the code runs!

In [39]:
using CUDA
gpu_v = CuArray([1,2,3,4,5,6,7,8]) #This array now lives on the GPU

#Some generic function
f₂(x) = x + 12 + x^2 - sin(x)

f₂.(gpu_v) # This is run on the GPU!!!

8-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 13.158529015192103
 17.09070257317432
 23.85887999194013
 32.75680249530793
 42.95892427466314
 54.27941549819893
 67.34301340128121
 83.01064175337662

I gotta see what this looks like in GPU assembly (warning, its filled with inlined code from the library)

In [40]:
using InteractiveUtils #We use the macro @code_native from here as its a bit easier to specify the function specialisation by a variable/call
show(@code_native f₂(gpu_v))

	.text
	.file	"f\342\202\202"
	.globl	"julia_f₂_7330"                 # -- Begin function julia_f₂_7330
	.p2align	4, 0x90
	.type	"julia_f₂_7330",@function
"julia_f₂_7330":                        # @"julia_f\E2\82\82_7330"
; ┌ @ /home/mjki2mb2/testJulia/test.ipynb:5 within `f₂`
	.cfi_startproc
# %bb.0:                                # %top
	subq	$24, %rsp
	.cfi_def_cfa_offset 32
; │┌ @ intfuncs.jl:340 within `literal_pow`
	movq	%rdi, 8(%rsp)
	movabsq	$139928263721120, %rax          # imm = 0x7F43967400A0
	movq	%rax, 16(%rsp)
	movabsq	$ijl_apply_generic, %rax
	movabsq	$139928054359488, %rdi          # imm = 0x7F4389F965C0
	leaq	8(%rsp), %rsi
	movl	$2, %edx
	callq	*%rax
	ud2
.Lfunc_end0:
	.size	"julia_f₂_7330", .Lfunc_end0-"julia_f₂_7330"
	.cfi_endproc
; └└
                                        # -- End function
	.section	".note.GNU-stack","",@progbits
nothing

# Symbolics

In [42]:
using Symbolics

#Here's a normal function
f₃(a, b) = a^2+b^2
display(f₃)

#I can use symbolic types to trace the function
@variables x(t), y(t), t
display(f₃(x,y))

f₃ (generic function with 1 method)

x(t)^2 + y(t)^2

How about discovering the governing differential equations behind data https://github.com/SciML/DataDrivenDiffEq.jl?
Global non-linear constrained optimisation (for small enough dimensionality) https://github.com/PSORLab/EAGO.jl

# Minimisation

Want global minimisation?

In [43]:
using IntervalArithmetic, IntervalOptimisation
const ∞ = Inf
minimise(x->sin(x)+(x-3)^2, -∞..∞)

([-0.102457, -0.101645], Interval{Float64}[[3.48339, 3.4844], [3.47441, 3.47495], [3.47231, 3.47284], [3.47335, 3.47389], [3.47388, 3.47442], [3.47023, 3.47077], [3.47127, 3.4718], [3.47546, 3.47599], [3.4765, 3.47703], [3.46971, 3.47024]  …  [3.49105, 3.49159], [3.45469, 3.45522], [3.45419, 3.4547], [3.45367, 3.4542], [3.49158, 3.49209], [3.49208, 3.4926], [3.45316, 3.45368], [3.49259, 3.49311], [3.45265, 3.45317], [3.4931, 3.49363]])

In [61]:
# EAGO needs some older code, so we can install it in its own environment, called "EAGO", to prevent the package manager downgrading some core functionality
using Pkg
Pkg.activate("EAGO") #Activate the EAGO environment
Pkg.add("JuMP")
Pkg.add("EAGO") # Install EAGO inside this environment

Pkg.activate() #Go back to the default environment

  Activating project at `~/testJulia/EAGO`
   Resolving package versions...


  No Changes to `~/testJulia/EAGO/Project.toml`
  No Changes to `~/testJulia/EAGO/Manifest.toml`
   Resolving package versions...


  No Changes to `~/testJulia/EAGO/Project.toml`
  No Changes to `~/testJulia/EAGO/Manifest.toml`


In [45]:
using JuMP._Derivatives
using JuMP, EAGO

m = Model(EAGO.Optimizer)

# Define bounded variables
xL = [10.0; 0.0; 0.0; 0.0; 0.0; 85.0; 90.0; 3.0; 1.2; 145.0]
xU = [2000.0; 16000.0; 120.0; 5000.0; 2000.0; 93.0; 95.0; 12.0; 4.0; 162.0]
@variable(m, xL[i] <= x[i=1:10] <= xU[i])

# Define nonlinear constraints
@NLconstraint(m, e1, -x[1]*(1.12+0.13167*x[8]-0.00667* (x[8])^2)+x[4] == 0.0)
@NLconstraint(m, e3, -0.001*x[4]*x[9]*x[6]/(98-x[6])+x[3] == 0.0)
@NLconstraint(m, e4, -(1.098*x[8]-0.038* (x[8])^2)-0.325*x[6]+x[7] == 57.425)
@NLconstraint(m, e5, -(x[2]+x[5])/x[1]+x[8] == 0.0)

# Define linear constraints
@constraint(m, e2, -x[1]+1.22*x[4]-x[5] == 0.0)
@constraint(m, e6, x[9]+0.222*x[10] == 35.82)
@constraint(m, e7, -3*x[7]+x[10] == -133.0)

# Define nonlinear objective
@NLobjective(m, Max, 0.063*x[4]*x[7] - 5.04*x[1] - 0.035*x[2] - 10*x[3] - 3.36*x[5])

# Solve the optimization problem
JuMP.optimize!(m)

-----------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes    | Lower Bound  |  Upper Bound  |      Gap     |     Ratio    |     Time     |    Time Left   |
-----------------------------------------------------------------------------------------------------------------------------


|         1000  |          315 |   -1.359E+03 |   -1.161E+03 |   1.979E+02 |    1.456E-01 |    7.854E+00 |    3.592E+03 |


|         2000  |          463 |   -1.278E+03 |   -1.161E+03 |   1.167E+02 |    9.131E-02 |    1.414E+01 |    3.586E+03 |


|         3000  |          571 |   -1.225E+03 |   -1.161E+03 |   6.385E+01 |    5.212E-02 |    2.086E+01 |    3.579E+03 |


|         4000  |          553 |   -1.198E+03 |   -1.161E+03 |   3.656E+01 |    3.052E-02 |    2.714E+01 |    3.573E+03 |


|         5000  |          491 |   -1.178E+03 |   -1.161E+03 |   1.667E+01 |    1.415E-02 |    3.388E+01 |    3.566E+03 |


|         6000  |          455 |   -1.171E+03 |   -1.161E+03 |   9.551E+00 |    8.157E-03 |    3.993E+01 |    3.560E+03 |


|         7000  |          437 |   -1.166E+03 |   -1.161E+03 |   5.163E+00 |    4.426E-03 |    4.563E+01 |    3.554E+03 |


|         8000  |          425 |   -1.164E+03 |   -1.161E+03 |   2.859E+00 |    2.456E-03 |    5.151E+01 |    3.548E+03 |


 
Relative Tolerance Achieved
First Solution Found at Node 141
LBD = 1161.324978726197
UBD = 1162.4739414697813
Solution is :
    X[1] = 1728.9208944688476
    X[2] = 15999.999997374796
    X[3] = 98.16067127564162
    X[4] = 3056.492536204471
    X[5] = 1999.9999997006068
    X[6] = 90.61609101391976
    X[7] = 94.1877648771026
    X[8] = 10.41111831932906
    X[9] = 2.616948591849675
    X[10] = 149.5632946313078
 
